In [1]:
%%configure -f
{ 
    "defaultLakehouse": { 
        "name":  "LH_CONFIGURATION"
    }
}

StatementMeta(, 8a578f63-c2d3-4b11-8787-5249a648642f, -1, Finished, Available, Finished)

In [38]:
# !!!!! create manually an sql database in the configuration workspace !!!!

# target connections guid 
# vul de juiste ID in voor de betreffende connectie

capacity_id = '560C908C-072A-478C-939C-CFC4398D2FDB' # Unique identifier for the capacity to be used.

workspace_roles = [ # Keep emtpy [] if you only want to assign this to your personal account
                    {
                        "principal": {
                            "id": "ba4a392b-9596-46d8-9486-53eb2c2b22bf",
                            "displayName": "sg-fabric-contributor",
                            "type": "Group"
                        },
                        "role": "Member"
                        },
                        {
                        "principal": {
                            "id": "39b98289-1540-4866-b9f0-3935f9eeebd6",
                            "displayName": "sg-fabric-admin",
                            "type": "Group"
                        },
                        "role": "Admin"
                        }
                    ]

logging = {
                    'workspace': {
                        'name' : 'FMD_FRAMEWORK_LOG_MH', # Name of target workspace
                        'roles' : workspace_roles,
                        'capacity_id' : capacity_id
                    },
                    'WarehouseName' : 'WH_LOG' # Name of target logging warehouse
}

configuration = {
                    'workspace': {
                        'name' : 'FMD_FRAMEWORK_CONFIG_MH', # Name of target workspace
                        'roles' : workspace_roles,
                        'capacity_id' : capacity_id
                    },
                    'DatabaseName' : 'SQL_FMD_FRAMEWORK' # Name of target configuration SQL Database
}

environments = [
                    {
                        'environment_name' : 'development', # Name of target environment
                        'workspaces': {
                            'data' : {
                                'name' : 'FMD_FRAMEWORK_DATA_MH3 (dvlm)', # Name of target code workspace for development
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id
                            },
                            'code' : {
                                'name' : 'FMD_FRAMEWORK_CODE_MH3 (dvlm)', # Name of target data workspace for development
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id
                            }
                        },
                        'connections' : {
                            'CON_FMD_FABRIC_API' : '644f217a-8761-4f19-b1b7-a40bfe0ec2af', # Required Guid to the Fabric API (web v2) connection (https://api.fabric.microsoft.com/v1/)
                            'CON_FMD_FABRIC_SQL' : '372237f9-709a-48f8-8fb2-ce06940c990e', # Required Guid to the Fabric SQL connection
                            'CON_FMD_FABRIC_PIPELINES' : '6d8146c6-a438-47df-94e2-540c552eb6d7', # Required Guid to the Fabric datapipelines connection
                            'CON_FMD_ASQL_01' : 'cf673e6a-13f6-4ebb-9cbb-4ba4ab390818', # Optional Guid to an Azure SQL database connection
                            'CON_FMD_ASQL_02' : '11a8e5fe-fbca-4822-9ba4-9162cf56e6dd', # Optional Guid to an second Azure SQL database connection
                            'CON_FMD_ADLS_01' : 'a0581b6e-5e38-46eb-bab2-7f08e9a35c30', # Optional Guid to an Azure SQL Datalake storage connection
                            'CON_FMD_ADF_PIPELINES' : 'e93f565a-e2bc-4b60-900e-1907e825e37c' # Optional Guid to an Azure Datafactory connection
                        }
                    },
                    {
                        'environment_name' : 'production', # Name of target environment
                        'workspaces': {
                            'data' : {
                                'name' : 'FMD_FRAMEWORK_DATA_MH (prod)',
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id
                            },
                            'code' : {
                                'name' : 'FMD_FRAMEWORK_CODE_MH (prod)',
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id
                            }
                        },
                        'connections' : {
                            'CON_FMD_FABRIC_API' : '644f217a-8761-4f19-b1b7-a40bfe0ec2af',
                            'CON_FMD_FABRIC_SQL' : '372237f9-709a-48f8-8fb2-ce06940c990e',
                            'CON_FMD_FABRIC_PIPELINES' : '6d8146c6-a438-47df-94e2-540c552eb6d7',
                            'CON_FMD_ASQL_01' : 'cf673e6a-13f6-4ebb-9cbb-4ba4ab390818',
                            'CON_FMD_ASQL_02' : '11a8e5fe-fbca-4822-9ba4-9162cf56e6dd',
                            'CON_FMD_ADLS_01' : 'a0581b6e-5e38-46eb-bab2-7f08e9a35c30',
                            'CON_FMD_ADF_PIPELINES' : 'e93f565a-e2bc-4b60-900e-1907e825e37c'
                        }
                    }
                ]

# source file to read the deployment manifest from (LH_CONFIGURATION) 
deployment_file = 'deployment/FMD_deployment.json'

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, 3, Finished, Available, Finished)

In [39]:
from json import loads, dumps
import json
import requests
import base64
import time
import uuid
import struct
import pyodbc

from typing import Callable, List, Dict, Optional, Any
from datetime import datetime
from time import sleep, time
from dataclasses import dataclass, field

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, 4, Finished, Available, Finished)

In [40]:
%run NB_FMD_DEPLOYMENT_UTILS

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, 8, Finished, Available, Finished)

In [41]:
tasks=[]

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, 9, Finished, Available, Finished)

In [42]:
fmd_api_access_token =  notebookutils.credentials.getToken('https://analysis.windows.net/powerbi/api')
fabric_session = create_fabric_session(fabric_token = fmd_api_access_token)

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, 10, Finished, Available, Finished)

In [43]:
# check if token is valid
for token in [fmd_api_access_token]:
    if not token:
        continue
    header, payload, signature = token.split('.')
    payload += '=' * (-len(payload) % 4)  # Add padding
    token_dict = loads(base64.urlsafe_b64decode(payload))
    directory_id = token_dict.get("tid")
    timest = token_dict.get("exp")
    expiry = (datetime.fromtimestamp(timest) - datetime.now()).total_seconds() // 60
    expiry_str = str(expiry) if expiry < 5 else str(expiry)
    print(F"token {directory_id} will expire in {expiry_str} minutes at\t{datetime.fromtimestamp(timest)} UTC")
print(F"Current time:\t\t\t\t\t\t\t\t\t{datetime.now().replace(microsecond=0)} UTC")

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, 11, Finished, Available, Finished)

token c183ff6a-8ca1-4185-9378-af980978377c will expire in 75.0 minutes at	2025-02-05 15:34:13 UTC
Current time:									2025-02-05 14:18:21 UTC


In [44]:
# Create necessary workspaces 
start = time()

for environment in environments:

    print(f"--------------------------")
    print(f"Processing: {environment['environment_name']}")
    
    # Loop through the workspace names and get their IDs
    
    for workspace in [environment['workspaces']['data'], environment['workspaces']['code'], configuration['workspace'], logging['workspace']]:
        
        print(f" -----")
        print(f" - Processing: data workspace {environment['environment_name']}")
        
        # List all workspaces
        workspaces_current = get_fabric_workspaces(fabric_session)        
        
        # Check if the displayName exists in the workspaces
        matching_workspaces = [workspace_current for workspace_current in workspaces_current.get('value') if workspace_current['displayName'] == workspace['name']]
        
        if matching_workspaces:
            print(f" - Workspace '{workspace['name']}' found. Workspace ID: {matching_workspaces[0]['id']}")
            workspace['id'] = matching_workspaces[0]['id']
        else:
            print(f" - Workspace '{workspace['name']}' not found. Creating new workspace...")
            workspace_created = fabric_request(fabric_session, F"workspaces/", 'POST', payload={"displayName": workspace['name']}, payloadtype='json')
            workspace['id'] = workspace_created['id']
            tasks.append({"task_name":f"create item {workspace['name']} initially", "task_duration": int(time() - start), "status": "success"})
        
        assign_fabric_workspace_capacity(fabric_session, workspace['id'], workspace['capacity_id'])
        tasks.append({"task_name":f"Workspace '{workspace['name']}' connected to capacity ID: {workspace['capacity_id']}", "task_duration": int(time() - start), "status": "success"})
        print(f" - Workspace '{workspace['name']}' created with ID: {workspace['id']} and capacity_id: {workspace['capacity_id']}")

        # Check if roles exists or create them
        print(f" - Assiging Workspace roles")
        assign_fabric_workspace_roles(fabric_session, workspace['id'], workspace['roles'])

    # Print the workspace IDs
    print(f"--------------------------")
    print(f"Workspace ID for data workspace {environment['environment_name']}: {environment['workspaces']['data']['id']}")
    print(f"Workspace ID for code workspace {environment['environment_name']}: {environment['workspaces']['code']['id']}")


StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, 12, Finished, Available, Finished)

--------------------------
Processing: development
 -----
 - Processing: data workspace development
 - Workspace 'FMD_FRAMEWORK_DATA_MH3 (dvlm)' found. Workspace ID: f898dfe0-f8c1-4fee-8e02-ddb81c19f2ac


Capacity found (560C908C-072A-478C-939C-CFC4398D2FDB)
Capacity (560C908C-072A-478C-939C-CFC4398D2FDB) assigning to workspace (f898dfe0-f8c1-4fee-8e02-ddb81c19f2ac)
Request accepted: Dont Wait for result
 - Workspace 'FMD_FRAMEWORK_DATA_MH3 (dvlm)' created with ID: f898dfe0-f8c1-4fee-8e02-ddb81c19f2ac and capacity_id: 560C908C-072A-478C-939C-CFC4398D2FDB
 - Assiging Workspace roles
Skipping: sg-fabric-contributor for f898dfe0-f8c1-4fee-8e02-ddb81c19f2ac
Skipping: sg-fabric-admin for f898dfe0-f8c1-4fee-8e02-ddb81c19f2ac
 -----
 - Processing: data workspace development
 - Workspace 'FMD_FRAMEWORK_CODE_MH3 (dvlm)' found. Workspace ID: 73eaa951-1faf-43d4-b800-79ddd94ea148


Capacity found (560C908C-072A-478C-939C-CFC4398D2FDB)
Capacity (560C908C-072A-478C-939C-CFC4398D2FDB) assigning to workspace (73eaa951-1faf-43d4-b800-79ddd94ea148)
Request accepted: Dont Wait for result
 - Workspace 'FMD_FRAMEWORK_CODE_MH3 (dvlm)' created with ID: 73eaa951-1faf-43d4-b800-79ddd94ea148 and capacity_id: 560C908C-072A-478C-939C-CFC4398D2FDB
 - Assiging Workspace roles
Skipping: sg-fabric-contributor for 73eaa951-1faf-43d4-b800-79ddd94ea148
Skipping: sg-fabric-admin for 73eaa951-1faf-43d4-b800-79ddd94ea148
 -----
 - Processing: data workspace development
 - Workspace 'FMD_FRAMEWORK_CONFIG_MH' found. Workspace ID: 5fed5713-5946-4c94-9906-1423da508c4e


Capacity found (560C908C-072A-478C-939C-CFC4398D2FDB)
Capacity (560C908C-072A-478C-939C-CFC4398D2FDB) assigning to workspace (5fed5713-5946-4c94-9906-1423da508c4e)
Request accepted: Dont Wait for result
 - Workspace 'FMD_FRAMEWORK_CONFIG_MH' created with ID: 5fed5713-5946-4c94-9906-1423da508c4e and capacity_id: 560C908C-072A-478C-939C-CFC4398D2FDB
 - Assiging Workspace roles
Skipping: sg-fabric-contributor for 5fed5713-5946-4c94-9906-1423da508c4e
Skipping: sg-fabric-admin for 5fed5713-5946-4c94-9906-1423da508c4e
 -----
 - Processing: data workspace development
 - Workspace 'FMD_FRAMEWORK_LOG_MH' found. Workspace ID: 56a439e6-c3b8-4fef-a92b-71a0a5bba22e


Capacity found (560C908C-072A-478C-939C-CFC4398D2FDB)
Capacity (560C908C-072A-478C-939C-CFC4398D2FDB) assigning to workspace (56a439e6-c3b8-4fef-a92b-71a0a5bba22e)
Request accepted: Dont Wait for result
 - Workspace 'FMD_FRAMEWORK_LOG_MH' created with ID: 56a439e6-c3b8-4fef-a92b-71a0a5bba22e and capacity_id: 560C908C-072A-478C-939C-CFC4398D2FDB
 - Assiging Workspace roles
Skipping: sg-fabric-contributor for 56a439e6-c3b8-4fef-a92b-71a0a5bba22e
Skipping: sg-fabric-admin for 56a439e6-c3b8-4fef-a92b-71a0a5bba22e
--------------------------
Workspace ID for data workspace development: f898dfe0-f8c1-4fee-8e02-ddb81c19f2ac
Workspace ID for code workspace development: 73eaa951-1faf-43d4-b800-79ddd94ea148
--------------------------
Processing: production
 -----
 - Processing: data workspace production
 - Workspace 'FMD_FRAMEWORK_DATA_MH (prod)' found. Workspace ID: 3695dee4-d689-43f8-a6cd-e92cfb0cdf8b


Capacity found (560C908C-072A-478C-939C-CFC4398D2FDB)
Capacity (560C908C-072A-478C-939C-CFC4398D2FDB) assigning to workspace (3695dee4-d689-43f8-a6cd-e92cfb0cdf8b)
Request accepted: Dont Wait for result
 - Workspace 'FMD_FRAMEWORK_DATA_MH (prod)' created with ID: 3695dee4-d689-43f8-a6cd-e92cfb0cdf8b and capacity_id: 560C908C-072A-478C-939C-CFC4398D2FDB
 - Assiging Workspace roles
Skipping: sg-fabric-contributor for 3695dee4-d689-43f8-a6cd-e92cfb0cdf8b
Skipping: sg-fabric-admin for 3695dee4-d689-43f8-a6cd-e92cfb0cdf8b
 -----
 - Processing: data workspace production
 - Workspace 'FMD_FRAMEWORK_CODE_MH (prod)' found. Workspace ID: 17194530-c51f-45b2-a745-88122009dcce


Capacity found (560C908C-072A-478C-939C-CFC4398D2FDB)
Capacity (560C908C-072A-478C-939C-CFC4398D2FDB) assigning to workspace (17194530-c51f-45b2-a745-88122009dcce)
Request accepted: Dont Wait for result
 - Workspace 'FMD_FRAMEWORK_CODE_MH (prod)' created with ID: 17194530-c51f-45b2-a745-88122009dcce and capacity_id: 560C908C-072A-478C-939C-CFC4398D2FDB
 - Assiging Workspace roles
Skipping: sg-fabric-contributor for 17194530-c51f-45b2-a745-88122009dcce
Skipping: sg-fabric-admin for 17194530-c51f-45b2-a745-88122009dcce
 -----
 - Processing: data workspace production
 - Workspace 'FMD_FRAMEWORK_CONFIG_MH' found. Workspace ID: 5fed5713-5946-4c94-9906-1423da508c4e


Capacity found (560C908C-072A-478C-939C-CFC4398D2FDB)
Capacity (560C908C-072A-478C-939C-CFC4398D2FDB) assigning to workspace (5fed5713-5946-4c94-9906-1423da508c4e)
Request accepted: Dont Wait for result
 - Workspace 'FMD_FRAMEWORK_CONFIG_MH' created with ID: 5fed5713-5946-4c94-9906-1423da508c4e and capacity_id: 560C908C-072A-478C-939C-CFC4398D2FDB
 - Assiging Workspace roles
Skipping: sg-fabric-contributor for 5fed5713-5946-4c94-9906-1423da508c4e
Skipping: sg-fabric-admin for 5fed5713-5946-4c94-9906-1423da508c4e
 -----
 - Processing: data workspace production
 - Workspace 'FMD_FRAMEWORK_LOG_MH' found. Workspace ID: 56a439e6-c3b8-4fef-a92b-71a0a5bba22e


Capacity found (560C908C-072A-478C-939C-CFC4398D2FDB)
Capacity (560C908C-072A-478C-939C-CFC4398D2FDB) assigning to workspace (56a439e6-c3b8-4fef-a92b-71a0a5bba22e)
Request accepted: Dont Wait for result
 - Workspace 'FMD_FRAMEWORK_LOG_MH' created with ID: 56a439e6-c3b8-4fef-a92b-71a0a5bba22e and capacity_id: 560C908C-072A-478C-939C-CFC4398D2FDB
 - Assiging Workspace roles
Skipping: sg-fabric-contributor for 56a439e6-c3b8-4fef-a92b-71a0a5bba22e
Skipping: sg-fabric-admin for 56a439e6-c3b8-4fef-a92b-71a0a5bba22e
--------------------------
Workspace ID for data workspace production: 3695dee4-d689-43f8-a6cd-e92cfb0cdf8b
Workspace ID for code workspace production: 17194530-c51f-45b2-a745-88122009dcce


In [45]:
# Read deployment manifest
deployment_manifest = {}
with open(f"{notebookutils.fs.getMountPath('/default')}/Files/{deployment_file}") as f:
    deployment_manifest = json.load(f)

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, 13, Finished, Available, Finished)

In [ ]:
# re-map databases

for target_item in deployment_manifest['logging']['items']:
    if target_item['type'] in ('Warehouse', 'SQLEndpoint'):
        target_item['displayName'] = logging['WarehouseName']

for target_item in deployment_manifest['configuration']['items']:
    if target_item['type'] in ('SQLDatabase', 'SQLEndpoint'):
        target_item['displayName'] = configuration['DatabaseName']

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, 14, Finished, Available, Finished)

In [64]:
def workspace_deployment(workspace_deployment_config, workspace_deployment_items, replace_collection, empty = True):
    print(f" --------------------------")
    print(f" Processing code workspace {workspace_deployment_config['name']}")
    
    target_items = get_fabric_items(fabric_session, workspace_deployment_config['id'])

    for deployment_item in workspace_deployment_items:
        for target_item in target_items['value']:
            if target_item['displayName'] == deployment_item['displayName'] \
                    and target_item['type'] == deployment_item['type']:
                print(f" - Skip existing: {deployment_item['displayName']}, {deployment_item['type']}, {target_item['id']}")
                break
        else:
            if deployment_item['type'] == 'SQLDatabase':
                raise Exception(f"Please create manually the SQL database {deployment_item['displayName']} in the configuration workspace {configuration['workspace']['name']} and re-run this code.")  
                # skip because sqldatabase isn't supported yet
            
            print(f" - Creating: {deployment_item['displayName']} {deployment_item['type']}")
            item = deployment_item.copy()
            
            if empty:
                if item.get('definition'):
                    print(f" - Dropping definition")
                    item.pop('definition')

            target_item = fabric_request(fabric_session, url=f"workspaces/{workspace_deployment_config['id']}/items/", method="POST", payload=item, payloadtype='json')

        deployment_item["new_id"] = target_item['id']

        if deployment_item['type'] in ('Warehouse', 'SQLDatabase'):
            if deployment_item.get('org_endpoint', '') != '':
                return_item = fabric_request(fabric_session, url=f"workspaces/{workspace_deployment_config['id']}/{deployment_item['type']}s/{target_item['id']}", method="GET")
                if deployment_item['type'] in ('Warehouse'):
                    if return_item.get("properties", {}).get("connectionString", "") != '':
                        deployment_item["connectionString"] = return_item["properties"]["connectionString"]
                        replace_collection.append({"old_id": deployment_item["org_endpoint"], "new_id": deployment_item["connectionString"]})
                
                if deployment_item['type'] in ('SQLDatabase'):
                    if return_item.get("properties", {}).get("serverFqdn", "") != '':
                        deployment_item["connectionString"] = return_item["properties"]["serverFqdn"].replace(',1433', '')
                        replace_collection.append({"old_id": deployment_item["org_endpoint"], "new_id": deployment_item["connectionString"]})
                    if return_item.get("properties", {}).get("databaseName", "") != '':
                        deployment_item["databaseName"] = return_item["properties"]["databaseName"]

        replace_collection.append({"old_id": deployment_item["org_id"], "new_id": deployment_item["new_id"]})

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, 15, Finished, Available, Finished)

In [65]:
# create empty items if not exists

for environment in environments:

    print(f"--------------------------")
    print(f"Processing: {environment['environment_name']}")

    environment['guids_to_replace'] = []
    workspace_deployment(environment['workspaces']['code'], deployment_manifest['items'], environment['guids_to_replace'], True)
    environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_code"], "new_id": environment['workspaces']['code']['id']})
    workspace_deployment(environment['workspaces']['data'], deployment_manifest['data'], environment['guids_to_replace'], True)
    environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_data"], "new_id": environment['workspaces']['data']['id']})
    workspace_deployment(logging['workspace'], deployment_manifest['logging']['items'], environment['guids_to_replace'], True)
    environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_logging"], "new_id": logging['workspace']['id']})
    workspace_deployment(configuration['workspace'], deployment_manifest['configuration']['items'], environment['guids_to_replace'], True)
    environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_config"], "new_id": configuration['workspace']['id']})

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, 16, Finished, Available, Finished)

--------------------------
Processing: development
 --------------------------
 Processing code workspace FMD_FRAMEWORK_CODE_MH3 (dvlm)
 - Skip existing: NB_FMD_LOAD_BRONZE_SILVER, Notebook, 1979757d-aa3e-423d-b491-91e0e881afeb
 - Skip existing: NB_FMD_LOAD_LANDING_BRONZE, Notebook, 93fedcab-37d5-4209-9252-4bc157e9899f
 - Skip existing: PL_FMD_LOAD_LANDINGZONE, DataPipeline, 3c6c5ddd-94af-4654-b59f-83872da0fee1
 - Skip existing: PL_FMD_LOAD_ALL, DataPipeline, c7813256-23e6-420d-9207-6da274afb0c6
 - Skip existing: PL_FMD_LOAD_BRONZE, DataPipeline, 087c609e-025f-4ac0-9b30-20a3376a2825
 - Skip existing: PL_FMD_LOAD_SILVER, DataPipeline, 25f59916-1095-4bbb-a5dc-648a0f798453
 - Skip existing: PL_FMD_LDZ_COPY_FROM_ASQL_01, DataPipeline, 324613db-976f-4f48-9b5e-5aa337fea879
 - Skip existing: PL_FMD_LDZ_COPY_FROM_ONELAKE_TABLES_01, DataPipeline, 979c1302-2d51-4c14-96bc-3023fc4d39b6
 - Skip existing: PL_FMD_LDZ_COPY_FROM_ADLS_01, DataPipeline, 4cb5d1f9-7bbe-4312-9c04-8916b0e629c7
 - Skip existi

KeyError: 'configuration'

In [48]:
items_to_deploy = deployment_manifest["items"]

for environment in environments:
    print(f"--------------------------")
    print(f"Processing: {environment['environment_name']}")
    
    # Append the remaining pairs
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_FABRIC_API"], "new_id": environment['connections']['CON_FMD_FABRIC_API']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_FABRIC_SQL"], "new_id": environment['connections']['CON_FMD_FABRIC_SQL']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_FABRIC_PIPELINES"], "new_id": environment['connections']['CON_FMD_FABRIC_PIPELINES']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ASQL_01"], "new_id": environment['connections']['CON_FMD_ASQL_01']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ASQL_02"], "new_id": environment['connections']['CON_FMD_ASQL_02']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ADLS_01"], "new_id": environment['connections']['CON_FMD_ADLS_01']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ADF_PIPELINES"], "new_id": environment['connections']['CON_FMD_ADF_PIPELINES']})
    
    # Deploy items to workspace
    existing_items = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['code']['id']}/items/", method="GET")
    deploy_items(items_to_deploy, environment['guids_to_replace'], fmd_api_access_token, environment['workspaces']['code']['id'], existing_items["value"])

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, -1, Cancelled, , Cancelled)

In [ ]:
driver = '{ODBC Driver 18 for SQL Server}'

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, -1, Cancelled, , Cancelled)

In [68]:
# by timeout re-run

for target_item in deployment_manifest['logging']['items']:
    if target_item['type'] == 'Warehouse':
        connstring = target_item["connectionString"]
        database = target_item['displayName']

try:
    i = 0
    token = mssparkutils.credentials.getToken('https://analysis.windows.net/powerbi/api').encode("UTF-16-LE")
    token_struct = struct.pack(f'<I{len(token)}s', len(token), token)
    print(f"DRIVER={driver};SERVER={connstring};PORT=1433;DATABASE={database};")
    connection = pyodbc.connect(f"DRIVER={driver};SERVER={connstring};PORT=1433;DATABASE={database};", attrs_before={1256:token_struct}, timeout=12)

    with connection.cursor() as cursor:
        cursor.execute("SELECT 1")  # Execute the warm-up query (a simple query like 'SELECT 1' can be used)
        cursor.fetchone()
        connection.timeout = 5  # Setting a lower timeout for subsequent queries
        for i, query in enumerate(deployment_manifest["logging"]["queries"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"success"})
except pyodbc.OperationalError as e:
    print(e) 
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"pyodbc failed: {e}"})
except Exception as e:
    print(e) 
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"failed: {e}"})

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, -1, Cancelled, , Cancelled)

In [ ]:
deployment_manifest["configuration"]["queries"].append(f'EXEC [integration].[sp_UpsertConnection] @ConnectionGuid = "00000000-0000-0000-0000-000000000000", @Name = "CON_FMD_ONELAKE", @Type = "ONELAKE", @IsActive = 1')
deployment_manifest["configuration"]["queries"].append(f'EXEC [integration].[sp_UpsertConnection] @ConnectionGuid = "{(environments[0]["connections"]["CON_FMD_ASQL_01"])}", @Name = "CON_FMD_ASQL_01", @Type = "SQL", @IsActive = 1')
deployment_manifest["configuration"]["queries"].append(f'EXEC [integration].[sp_UpsertConnection] @ConnectionGuid = "{(environments[0]["connections"]["CON_FMD_ASQL_02"])}", @Name = "CON_FMD_ASQL_02", @Type = "SQL", @IsActive = 1')
deployment_manifest["configuration"]["queries"].append(f'EXEC [integration].[sp_UpsertConnection] @ConnectionGuid = "{(environments[0]["connections"]["CON_FMD_ADLS_01"])}", @Name = "CON_FMD_ADLS_01", @Type = "ADLS", @IsActive = 1')
deployment_manifest["configuration"]["queries"].append(f'EXEC [integration].[sp_UpsertConnection] @ConnectionGuid = "{(environments[0]["connections"]["CON_FMD_ADF_PIPELINES"])}", @Name = "CON_FMD_ADF_PIPELINES", @Type = "ADF", @IsActive = 1')

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, -1, Cancelled, , Cancelled)

In [ ]:
deployment_manifest["configuration"]["queries"].append("""
    DECLARE @DataSourceIdInternal INT = (SELECT DataSourceId FROM integration.DataSource WHERE Name = 'LH_DATA_LANDINGZONE')
    DECLARE @ConnectionIdInternal INT = (SELECT ConnectionId FROM integration.Connection WHERE ConnectionGuid = '00000000-0000-0000-0000-000000000000')
    EXECUTE [integration].[sp_UpsertDataSource] 
        @ConnectionId = @ConnectionIdInternal
        ,@DataSourceId = @DataSourceIdInternal
        ,@Name = 'LH_DATA_LANDINGZONE'
        ,@Abbreviation = 'ONELAKE'
        ,@Type = 'ONELAKE_TABLES_01'
        ,@Description = 'ONELAKE_TABLES'
        ,@IsActive = 1
""")

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, -1, Cancelled, , Cancelled)

In [ ]:
workspaces = []
workspaces.append(logging['workspace'])
workspaces.append(configuration['workspace'])

for environment in environments:
    workspaces.append(environment['workspaces']['code'])
    workspaces.append(environment['workspaces']['data'])
    
for workspace in workspaces:
    print(f'EXEC [integration].[sp_UpsertWorkspace](@WorkspaceId = "{workspace["id"]}" ,@Name = "{workspace["name"]}")')
    deployment_manifest["configuration"]["queries"].append(f'EXEC [integration].[sp_UpsertWorkspace] @WorkspaceId = "{workspace["id"]}", @Name = "{workspace["name"]}"')

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, -1, Cancelled, , Cancelled)

In [ ]:
for environment in environments:
    existing_items = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['code']['id']}/items/", method="GET")
    for item in existing_items.get('value', []):
        if item['type'] == 'DataPipeline':
            print(f'EXEC [integration].[sp_UpsertPipeline] @PipelineId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')
            deployment_manifest["configuration"]["queries"].append(f'EXEC [integration].[sp_UpsertPipeline] @PipelineId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, -1, Cancelled, , Cancelled)

In [ ]:
for environment in environments:
    existing_items = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['data']['id']}/items/", method="GET")
    for item in existing_items.get('value', []):
        if item['type'] == 'Lakehouse':
            print(f'EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')
            deployment_manifest["configuration"]["queries"].append(f'EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, -1, Cancelled, , Cancelled)

In [ ]:
deployment_manifest["configuration"]["queries"].append("""
    DECLARE @LandingzoneEntityIdInternal INT = (SELECT LandingzoneEntityId FROM integration.LandingzoneEntity WHERE SourceSchema = 'in' and SourceName = 'customer')
    DECLARE @DataSourceIdInternal INT = (SELECT DataSourceId FROM integration.DataSource WHERE Name = 'LH_DATA_LANDINGZONE')
    DECLARE @LakehouseIdInternal INT = (SELECT top 1 LakehouseId FROM integration.Lakehouse WHERE Name = 'LH_LANDINGZONE')
    EXECUTE [integration].[sp_UpsertLandingzoneEntity] 
        @LandingzoneEntityId = @LandingzoneEntityIdInternal
        ,@DataSourceId = @DataSourceIdInternal
        ,@LakehouseId = @LakehouseIdInternal
        ,@SourceSchema = 'in'
        ,@SourceName = 'customer'
        ,@FileName = 'customer'
        ,@FilePath = 'in'
        ,@FileType = 'parquet'
        ,@IsIncremental = 0
        ,@IsIncrementalColumn = ''
        ,@IsActive = 1
""")

In [ ]:
deployment_manifest["configuration"]["queries"].append("""
    DECLARE @LandingzoneEntityIdInternal INT = (SELECT LandingzoneEntityId FROM integration.LandingzoneEntity WHERE SourceSchema = 'in' and SourceName = 'customer')
    DECLARE @BronzeLayerEntityIdInternal INT = (SELECT BronzeLayerEntityId FROM integration.BronzeLayerEntity WHERE [Schema] = 'in' and [Name] = 'customer')
    DECLARE @LakehouseIdInternal INT = (SELECT top 1 LakehouseId FROM integration.Lakehouse WHERE Name = 'LH_BRONZE_LAYER')
    EXECUTE [integration].[sp_UpsertBronzeLayerEntity] 
        @BronzeLayerEntityId = @BronzeLayerEntityIdInternal
        ,@LandingzoneEntityId = @LandingzoneEntityIdInternal
        ,@Schema = 'in'
        ,@Name = 'customer'
        ,@FileType = 'Delta'
        ,@LakehouseId = @LakehouseIdInternal
        ,@PrimaryKeys = 'CustomerId'
        ,@IsActive = 1
""")

In [ ]:
deployment_manifest["configuration"]["queries"].append("""
    DECLARE @BronzeLayerEntityIdInternal INT = (SELECT BronzeLayerEntityId FROM integration.BronzeLayerEntity WHERE [Schema] = 'in' and [Name] = 'customer')
    DECLARE @SilverLayerEntityIdInternal INT = (SELECT SilverLayerEntityId FROM integration.SilverLayerEntity WHERE [Schema] = 'in' and [Name] = 'customer')
    DECLARE @LakehouseIdInternal INT = (SELECT top 1 LakehouseId FROM integration.Lakehouse WHERE Name = 'LH_SILVER_LAYER')
    EXECUTE [integration].[sp_UpsertSilverLayerEntity] 
        @SilverLayerEntityId = @SilverLayerEntityIdInternal
        ,@BronzeLayerEntityId = @BronzeLayerEntityIdInternal
        ,@LakehouseId = @LakehouseIdInternal
        ,@Name = 'customer'
        ,@Schema = 'in'
        ,@FileType = 'delta'
        ,@IsActive = 1
""")

In [70]:
for target_item in deployment_manifest['configuration']['items']:
    if target_item['type'] == 'SQLDatabase':
        connstring = target_item["connectionString"]
        database = target_item['databaseName']

try:
    i = 0
    token = mssparkutils.credentials.getToken('https://analysis.windows.net/powerbi/api').encode("UTF-16-LE")
    token_struct = struct.pack(f'<I{len(token)}s', len(token), token)
    print(f"DRIVER={driver};SERVER={connstring};PORT=1433;DATABASE={database};")
    connection = pyodbc.connect(f"DRIVER={driver};SERVER={connstring};PORT=1433;DATABASE={database};", attrs_before={1256:token_struct}, timeout=12)

    with connection.cursor() as cursor:
        cursor.execute("SELECT 1")  # Execute the warm-up query (a simple query like 'SELECT 1' can be used)
        cursor.fetchone()
        connection.timeout = 5  # Setting a lower timeout for subsequent queries
        for i, query in enumerate(deployment_manifest["configuration"]["queries"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"success"})
except pyodbc.OperationalError as e:
    print(e) 
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"pyodbc failed: {e}"})
except Exception as e:
    print(e) 
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"failed: {e}"})

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, -1, Cancelled, , Cancelled)

In [ ]:
display(tasks)

StatementMeta(, 497e73d3-8af1-4695-9e28-32a07c3c6c03, -1, Cancelled, , Cancelled)